In [37]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
ORG_NAME = 'w3b3d3v'

def get_github_data(org_name, access_token):
    url = f"https://api.github.com/orgs/{org_name}/repos"
    headers = {'Authorization': f'token {access_token}'}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the JSON response
        repos = response.json()
        df = pd.DataFrame(repos)
        # print(df)
        return df
        for repo in repos:
            print("repo: ", repo)
            # print(f"Repository Name: {repo['name']}")
            # print(f"Stars: {repo['stargazers_count']}")
            # print(f"Forks: {repo['forks_count']}")
            # print(f"Open Issues: {repo['open_issues_count']}")
            print("--------------------------------------------------")
    else:
        print("Failed to retrieve data:", response.status_code, response.text)

# Replace 'your_org_name' with your organization's name and 'your_access_token' with your actual GitHub access token

In [53]:
def fetch_issues_and_assignees(org_name, access_token):
    # Endpoint da API GraphQL do GitHub
    url = 'https://api.github.com/graphql'
    
    # Cabeçalho com o token de acesso
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    
    # Query GraphQL modificada para incluir createdAt e closedAt
    query = """
    {
      organization(login: "%s") {
        repositories(first: 100) {
          nodes {
            name
            issues(first: 100, states: [OPEN, CLOSED]) {
              nodes {
                number
                title
                state
                createdAt
                closedAt
                author {
                  login
                }
                assignees(first: 10) {
                  nodes {
                    login
                  }
                }
              }
            }
          }
          pageInfo {
            endCursor
            hasNextPage
          }
        }
      }
    }
    """ % org_name

    # Fazendo o request para a API
    response = requests.post(url, json={'query': query}, headers=headers)
    data = response.json()

    # Extrair dados para o DataFrame
    rows = []
    for repo in data['data']['organization']['repositories']['nodes']:
        repo_name = repo['name']
        for issue in repo['issues']['nodes']:
            for assignee in issue['assignees']['nodes']:
                rows.append({
                    'Repository': repo_name,
                    'Issue Number': issue['number'],
                    'Issue Title': issue['title'],
                    'Issue State': issue['state'],
                    'Issue Created At': issue['createdAt'],
                    'Issue Closed At': issue['closedAt'],
                    'Issue Author': issue['author']['login'],
                    'Assignee': assignee['login']
                })
    
    # Criando o DataFrame
    df = pd.DataFrame(rows)
    return df

In [54]:
# Substitua 'your_org_name' e 'your_access_token' com seus valores reais
dataframe = fetch_issues_and_assignees(ORG_NAME, GITHUB_TOKEN)

In [55]:
len(dataframe)

776

In [48]:
# dataframe[dataframe["Issue State"] == "OPEN"]
dataframe["Assignee"].value_counts()

Assignee
annabida                        82
allluz                          77
nomadbitcoin                    74
viniblack                       71
s4mp41xao                       70
JennyT3                         59
mikaelcarrara                   55
heronlancellot                  35
JoaoPFS0                        27
andressa-cabral                 23
Rodrigosrfm                     23
guilhermeboaventurarodrigues    22
RafaDSan                        19
0xneves                         17
marcaum-eth                     15
durdsvianna                     14
danicuki                        13
JABAGAMEHUNTER                  11
Hardsys-Andre                   10
AIengineer13                     9
Guboehm                          8
duudaheckker                     8
diegosano                        8
cavazinf                         7
junowoz                          4
piadascripto                     4
banafederico                     4
Jaxiii                           3
andersonlth